In [2]:
import keras 
import numpy as np
from keras.preprocessing.text import Tokenizer
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Input, Dense, Dropout, Embedding, LSTM, Flatten, GlobalAveragePooling1D, LSTM, Reshape,Conv1D
from keras.models import Model, Sequential
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
import itertools
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

Using TensorFlow backend.


In [2]:
%matplotlib inline
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [76]:
# train_data = pd.read_csv('text_emotion.csv')
train_data = pd.read_csv('emotion_classification.csv')

In [77]:
train_data.head()

,content,sentiment_value
0,it makes me so fucking irate jesus. nobody is ...,0.750
1,Lol Adam the Bull with his fake outrage...,0.417
2,@THATSSHAWTYLO passed away early this morning ...,0.354
3,@Kristiann1125 lol wow i was gonna say really?...,0.438
4,I need a \xf0\x9f\x8d\xb1sushi date\xf0\x9f\x8...,0.271


In [78]:
# num_class = len(np.unique(train_data.sentiment.values))
y = train_data['sentiment_value'].values
# num_class
# np.unique(train_data.sentiment.values)

In [79]:
MAX_LENGTH = 100
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_data.content.values)
post_seq = tokenizer.texts_to_sequences(train_data.content.values)
post_seq_padded = pad_sequences(post_seq, maxlen=MAX_LENGTH)

In [124]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
y = y.reshape(-1, 1)
y = sc.fit_transform(y)
# scalarX.fit(post_seq_padded)
# scalarY.fit(y.reshape(1700,1))
X_train, X_test, y_train, y_test = train_test_split(post_seq_padded, y, test_size=0.05)
X_train

array([[    0,     0,     0, ...,     1,  8443,   674],
       [    0,     0,     0, ...,   657,   102, 12667],
       [    0,     0,     0, ...,    40,    40,   809],
       ...,
       [    0,     0,     0, ...,    25,   554,   146],
       [    0,     0,     0, ...,   271,   155,   701],
       [    0,     0,     0, ...,   140,   321,   917]])

In [84]:
vocab_size = len(tokenizer.word_index) + 1

In [118]:
inputs = Input(shape=(MAX_LENGTH, ))
embedding_layer = Embedding(vocab_size,
                            128,
                            input_length=MAX_LENGTH)(inputs)
# # x = Dropout(0.1)(x)
# x = Flatten()(embedding_layer)
# # x = LSTM(64, input_shape=(32, 32), return_sequences=True)(embedding_layer)
# # # x = Conv1D(128, 5, activation='relu')(embedding_layer)
# # x = LSTM(64, input_shape=(32, 32), return_sequences=True)(x)
# # # x = Dropout(0.1)(x)
# # x = LSTM(32, input_shape=(32, 32))(x)
# # # x = Dropout(0.1)(x)
# x = Dense(64, input_dim=1,  kernel_initializer = 'uniform', activation='relu')(x)
# x = Dense(32,  kernel_initializer = 'uniform', activation='relu')(x)
# x = Dense(16,  kernel_initializer = 'uniform', activation='relu')(x)
# x = Dropout(0.1)(x)
model = Sequential()
model.add(Dense(50, input_dim=100, kernel_initializer = 'uniform', activation='relu'))
model.add(Dense(50, kernel_initializer = 'uniform', activation='relu'))
model.add(Dense(1, kernel_initializer='lecun_normal', activation='linear'))

# predictions = Dense(1, kernel_initializer='lecun_normal', activation='linear')(x)
# model = Model(inputs=[inputs], outputs=predictions)
model.compile(optimizer='adam',
              loss='mean_squared_error',
              metrics=['mae'])

model.summary()
filepath="weights-simple.hdf5"
checkpointer = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True)
history = model.fit(X_train, batch_size=64, y=y_train, verbose=1, validation_split=0.25, 
          shuffle=True, epochs=10, callbacks=[checkpointer])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_99 (Dense)             (None, 50)                5050      
_________________________________________________________________
dense_100 (Dense)            (None, 50)                2550      
_________________________________________________________________
dense_101 (Dense)            (None, 1)                 51        
Total params: 7,651
Trainable params: 7,651
Non-trainable params: 0
_________________________________________________________________
Train on 5057 samples, validate on 1686 samples
Epoch 1/10
5057/5057 [==============================] - 2s 457us/step - loss: 105.6879 - mean_absolute_error: 5.5127 - val_loss: 15.0983 - val_mean_absolute_error: 2.6630
Epoch 2/10
5057/5057 [==============================] - 0s 26us/step - loss: 10.0073 - mean_absolute_error: 2.2313 - val_loss: 9.9811 - val_mean_absolute_error: 2.1830
Epoch 3/10
5057/5

In [10]:
df = pd.DataFrame({'epochs':history.epoch, 'accuracy': history.history['acc'], 'validation_accuracy': history.history['val_acc']})
g = sns.pointplot(x="epochs", y="accuracy", data=df, fit_reg=False)
g = sns.pointplot(x="epochs", y="validation_accuracy", data=df, fit_reg=False, color='green')

NameError: name 'history' is not defined

In [119]:
predicted = model.predict(X_test)
predicted = np.argmax(predicted, axis=1)
# accuracy_score(y_test, predicted)

In [120]:
y_test

array([[ 0.62740567],
       [ 0.11146153],
       [-0.64181692],
       [ 1.33940858],
       [ 1.07627707],
       [-0.5644253 ],
       [-0.22390217],
       [-1.647908  ],
       [ 1.93790379],
       [ 0.7563917 ],
       [ 0.21980979],
       [-0.09491613],
       [ 0.11146153],
       [-0.52314977],
       [-0.75016519],
       [-0.10523501],
       [ 0.7563917 ],
       [-1.44153034],
       [ 0.32299862],
       [ 0.7563917 ],
       [ 0.00311326],
       [ 0.86473997],
       [-1.07005056],
       [ 1.95854155],
       [-1.17839883],
       [-1.2867471 ],
       [-1.39509537],
       [-0.21358328],
       [ 0.53969516],
       [-0.64181692],
       [-0.53346865],
       [ 0.00311326],
       [ 0.21980979],
       [ 1.18462534],
       [-1.50344364],
       [ 1.40132188],
       [ 1.72120725],
       [ 1.29297361],
       [-0.82239737],
       [-0.10523501],
       [ 2.27326748],
       [ 1.69541004],
       [ 0.53969516],
       [ 0.32299862],
       [ 1.40132188],
       [ 1

In [121]:
predicted

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [102]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

In [18]:
cnf_matrix = confusion_matrix(y_test, predicted)
np.set_printoptions(precision=2)

ValueError: Classification metrics can't handle a mix of continuous and binary targets

In [19]:
class_names = ['anger_disgust', 'fear', 'joy','sadness', 'shame_guilt']
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_names,
                      title='Confusion matrix, without normalization')

NameError: name 'cnf_matrix' is not defined

<Figure size 432x288 with 0 Axes>

In [20]:
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_names, normalize=True,
                      title='Normalized confusion matrix')

NameError: name 'cnf_matrix' is not defined

<Figure size 432x288 with 0 Axes>

In [21]:
tweets = [
  "Watching the sopranos again from start to finish!",
  "Finding out i have to go to the  dentist tomorrow",
  "Sun in my eyes but I don't mind, what a beautiful day we've had in New York today!",
  "Feels like someone's stabbed me in my hope",
  "Do people have no Respect for themselves or you know others peoples homes",
  "I want to go outside and chalk but I have no chalk",
  "I hate coming to the doctors when I feel as if I might have a big problem",
  "My mom wasn't mad",
  "You don't indicate once I'm already in the road THEN rev and honk at me you stupid bitch #learnhowtodrive #bitch",
  "Come home from work and this is on my doorstep. I guess he has a secret admirer",
  "The 'egyption hot models' facebook page is pathetic... simply photos of obese horny women.",
  "I HATE PAPERS AH #AH #HATE",
]

post = """West Bengal chief minister and Trinamool Congress (TMC) chief Mamata Banerjee's efforts to rename the state as 'Bangla' have again faced an obstacle from the Ministry of Home Affairs (MHA).

According to The Indian Express, the home ministry has expressed concern over this move, saying that the name "may sound like Bangladesh and that it would be difficult to differentiate the two at international forums". The home ministry has written to the Ministry of External Affairs, the report said, to obtain an opinion before any further consideration.

However, in 2016, Mamata had addressed the possibility of the name being confused with Bangladesh. Noting that the name 'Bangla' had historical significance, she had said, "In English it will be Bengal, so that there will be no confusion with the name of neighbouring Bangladesh."

File image of West Bengal chief minister Mamata Banerjee. PTIFile image of West Bengal chief minister Mamata Banerjee. PTI
A constitutional amendment is required for a change in the name of a state. A recent example of the name of a state being changed was that of Odisha (earlier called Orissa).

Banerjee's plans to rename the state have been taking shape since 2011, when the newly elected chief minister had put forward 'Paschim Banga' as an option.

Banerjee's effort in 2011 never saw the light of day at the Centre, and was unsuccessful even in 2016 when the state Assembly passed a resolution to change the name of West Bengal to language-specific names. The cabinet had proposed that the name of the state be 'Bengal' in English, 'Bangla' in Bengali, and 'Bangal' in Hindi. Then, the home ministry had rejected the proposal saying that it would not be possible for a state to have different names in different languages.

However, in July this year, the West Bengal Assembly unanimously passed a resolution to change the state's name to 'Bangla'.

According to reports, the state's Assembly pushed more strongly for the renaming after the July 2016 Inter-State Council meeting in Delhi, when Banerjee was the last chief minister to address the Council because the speakers were listed in alphabetical order of the states. The name 'Bangla' would bump the state up in an alphabetically-organised list to number four.

Apart from political repercussions of the move to rename the state, with the Left, Congress, and BJP state parties walking out of the Assembly in August 2016 over the issue, political analysts have warned against reigniting tensions in the Darjeeling and Kalimpong areas of the state. People residing in "the hills" as the areas are colloquially referred to, had in 2017 initiated a strong resistance to the state government's move to make Bengali a mandatory language in schools in the state.

The Gorkha and Nepali communities of the state demanded a separate state "Gorkhaland" in protest and as an assertion of the linguistic differences in the state. Even though those protests were successfully pacified by the Mamata Banerjee-government after three months, experts believe that there continues to be a divide in socio-cultural aspects in the state.

As this report by Marcus Dam points out, the attempt to rename the state may not go down well in the hills, as "Bangla" is also the locally accepted term for the Bengali language."""

list = post.split(".")

post_seq = tokenizer.texts_to_sequences(list)
post_seq_padded = pad_sequences(post_seq, maxlen=MAX_LENGTH)
p = model.predict(post_seq_padded)
p = np.argmax(p, axis=1)
final = [0, 0, 0, 0]
for i in p:
    final[i] += 1

final
# for i, l in enumerate(list):
#     print(l, " : ", p[i])

[22, 0, 0, 0]

In [9]:
import pandas
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import Ridge
from scipy.sparse import hstack
from sklearn.metrics import mean_squared_error
from sklearn.externals import joblib

def get_data(filename):
    df = pandas.read_csv(filename)
    df['content'] = df['content'].str.lower().replace('[^a-zA-Z0-9]', ' ', regex = True)
    return df

train_fn = 'emotion_classification.csv'
# test_fn = 'salary-test-mini.csv'

train = get_data(train_fn)
# # test = get_data(test_fn)

vectorizer = TfidfVectorizer(min_df=5)
# enc = DictVectorizer()
# clf = Ridge(alpha=1.0, random_state=241,)
# # train data set
X_train = vectorizer.fit_transform(train['content'])
print(X_train.shape)
# X_train = hstack([X_train])

# X_train, X_test, y_train, y_test = train_test_split(X_train, train['sentiment_value'], test_size=0.05)

# # train the model
# clf.fit(X_train, y_train)
# rslt = clf.predict(X_test)
# score = np.sqrt(mean_squared_error(rslt, y_test))
# filename = 'intensity_anger.sav'
# joblib.dump(clf, filename)
# vect = joblib.dump(vectorizer, 'dump.sav')

loaded_model = joblib.load("intensity_anger.sav")
vect = joblib.load('dump.sav')
X_train = vect.transform(["fuck off mother fucker dont fuck with me"])
X_train.shape
# X_train = hstack([X_train])
# print(X_train)
# result = loaded_model.predict(X_train)
# result
# score = np.sqrt(mean_squared_error(rslt, y_test))
# print(score)

(7098, 2434)


(1, 2434)